In [2]:
from mr_eval.utils.utils import *
import uuid

In [11]:
# First combine all qwq generated answers into one file:
files = [
    "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/build_data/selection_of_data/new_8_classes/one_question_multi_answer/prm_test_p1_qwq.jsonl",
    "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/build_data/selection_of_data/new_8_classes/one_question_multi_answer/prm_test_p2_qwq.jsonl"
]
output_file = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/annotation/data/multi_solutions/prm_test12_qwq.jsonl"

output_data = []
for input_file in files:
    a = process_jsonl(input_file)
    output_data.extend(a)

cleaned_data = []
for item in output_data:
    if item["modified_process"] == [] or len(item["modified_process"]) == 0:
        continue
    cleaned_data.append(item)
print(f"Total number of examples: {len(cleaned_data)}")
write_jsonl(cleaned_data, output_file)

Total number of examples: 174


In [12]:
# then load from the combined file and generate the final qwq file
input_data = process_jsonl(output_file)
tobe_annotated_file = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/annotation/data/multi_solutions/prm_test12_qwq_tobe_annotated.jsonl"

prompt="""
### Requirements
Please annotate whether the solution procedure provided is correct or not.
Please annotate whether the solution remains the same with the original question or not.
"""

def process_str_to_label_obj(string):
    res_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, string))
    res =  dict(
        message_id = res_id,
        content=string,
        message_type = "receive",
        user_id = "",
        parent_id = None, 
    )
    return res

In [13]:
output_data = []
for item in input_data:
    original_process = item["original_process"]
    modified_process = item["modified_process"]
    new_original_process = []
    new_modified_process = []
    for idx,step in enumerate(original_process):
        step = f"{idx+1}. {step}"
        new_original_process.append(step)
    
    for idx,step in enumerate(modified_process):
        step = f"{idx+1}. {step}"
        new_modified_process.append(step)
    
    original_process = "## Origin Process \n\n" + "\n\n".join(new_original_process)
    modified_process = "## Modified Process \n\n" + "\n\n".join(new_modified_process)
    
    # modified_steps = "## Modified Steps \n\n" + str(item["modified_steps"])
    # hallucination_types = "## Hallucination Types \n\n" + str(item["hallucination_types"])
    # hallucination_steps = "## Hallucination Steps \n\n" + str(item["hallucination_steps"])
    # reason = "## Reason \n\n" + str(item["reason"])
    
    # step_conv = modified_steps + "\n\n" + hallucination_steps + "\n\n" + hallucination_types + "\n\n" + reason
    question = item["question"]
    ground_truth = item["ground_truth"]
    
    question = process_str_to_label_obj(f"###Question:\n\n{question}")
    ground_truth = process_str_to_label_obj(f"###Ground Truth:\n\n{ground_truth}")
    original_process = process_str_to_label_obj(original_process)
    modified_process = process_str_to_label_obj(modified_process)
    # step_conv = process_str_to_label_obj(step_conv)
    conv_list = [question, ground_truth, original_process, modified_process, ]
    # print(conversation)
    # prompt="请判断LM给出的改动是否合理"
    output_data.append({"prompt": prompt, "conversation": conv_list,"custom":item})

In [14]:
write_jsonl(output_data, tobe_annotated_file)

In [8]:
## After annotation, we need to combine the annotated data into the original data file

annotation_file = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/annotation/data/multi_solutions/prm_testp12_qwq_annotations.jsonl"
classification = "multi_solutions"
annotated_data = process_jsonl(annotation_file)

In [9]:
filtered_data = []
for item in annotated_data:
    if item["evaluation"]["conversation_evaluation"]["correct"] == "true" \
        and item["evaluation"]["conversation_evaluation"]["same"] == "false":
        item["custom"]["classification"] = classification
        filtered_data.append(item["custom"])

print(f"Total number of annotated examples: {len(filtered_data)}")

Total number of annotated examples: 165


In [10]:
output_file="/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_annotate/build_data/selection_of_data/new_8_classes/one_question_multi_answer/prm_test_p12_multi_solutions.jsonl"
write_jsonl(filtered_data, output_file)